In [1]:
!nvidia-smi

Tue Mar 14 15:21:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0%   43C    P8    25W / 460W |      5MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1C:00.0 Off |                  N/A |
| 27%   

In [2]:
!pip install torch
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 17.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 74.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import torch

from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig


from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple
from sklearn.model_selection import train_test_split
import gc
from tqdm.auto import tqdm, trange


import pandas as pd
from sklearn.utils import shuffle

from collections import Counter
from tqdm import tqdm


# model

In [4]:
!nvidia-smi

Tue Mar 14 15:23:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:1B:00.0 Off |                  Off |
|  0%   41C    P8    25W / 460W |      5MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:1C:00.0 Off |                  N/A |
| 27%   

In [5]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [6]:
if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')
    
    
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

GPU


In [7]:
from transformers import TrainingArguments, Trainer

model_checkpoint = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

# **Read data SemEval2018-Task9**

In [7]:
import os
path = os.getcwd().replace('hearst_patterns', '')

In [8]:
path_data_en = path+"SemEval2018-Task9/training/data/1A.english.training.data.txt"
path_gold_en = path+"SemEval2018-Task9/training/gold/1A.english.training.gold.txt"

train_data_en_data = pd.read_csv(path_data_en, header=None, sep="\t", names=['term', 'relation'])
train_gold_en_data = pd.read_csv(path_gold_en, header=None, names=['hypernym'])

train_data_en_data.head()

,term,relation
0,blackfly,Concept
1,Turonian,Entity
2,abhorrence,Concept
3,tropical storm,Concept
4,militarization,Concept


In [9]:
path_test_data_en = path+"SemEval2018-Task9/test/data/1A.english.test.data.txt"
path_test_gold_en = path+"SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

test_data_en_data = pd.read_csv(path_test_data_en, header=None, sep="\t", names=['term', 'relation'])
test_gold_en_data = pd.read_csv(path_test_gold_en, header=None, names=['hypernym'])

In [14]:
hearst_patterns = """
There are a lot of [PARENT] such as

There were a lot of [PARENT] such as

There are a lot of [PARENT] here such as

Other [PARENT] such as

My favorite [PARENT] is either

There were a lot of [PARENT] here such as

which includes various [PARENT] like

Other [PARENT] especially

which includes various [PARENT] such as

My favorite [PARENT] is

I know such types of [PARENT] as

I know such kinds of [PARENT] as

[PARENT] such as

I know many kinds of [PARENT] for example

Other [PARENT] for example

[PARENT] namely

I know many types of [PARENT] for example

[PARENT] including

There are a lot of [PARENT] for example

which includes various [PARENT] for example

There are a lot of [PARENT] here for example

[PARENT] e.g.

[PARENT] like

[PARENT] especially

[PARENT] for example

[PARENT] for instance
"""

hearst_patterns = [pattern for ind, pattern in enumerate(hearst_patterns.split('\n')) if ind % 2 != 0]  
hearst_patterns

['There are a lot of [PARENT] such as',
 'There were a lot of [PARENT] such as',
 'There are a lot of [PARENT] here such as',
 'Other [PARENT] such as',
 'My favorite [PARENT] is either',
 'There were a lot of [PARENT] here such as',
 'which includes various [PARENT] like',
 'Other [PARENT] especially',
 'which includes various [PARENT] such as',
 'My favorite [PARENT] is',
 'I know such types of [PARENT] as',
 'I know such kinds of [PARENT] as',
 '[PARENT] such as',
 'I know many kinds of [PARENT] for example',
 'Other [PARENT] for example',
 '[PARENT] namely',
 'I know many types of [PARENT] for example',
 '[PARENT] including',
 'There are a lot of [PARENT] for example',
 'which includes various [PARENT] for example',
 'There are a lot of [PARENT] here for example',
 '[PARENT] e.g.',
 '[PARENT] like',
 '[PARENT] especially',
 '[PARENT] for example',
 '[PARENT] for instance']

In [72]:
def hearest_preprocessing(train_features, train_target, test_features, test_target, hearst_pattern):
    hearst_pattern = hearst_pattern.replace('[PARENT]', '<extra_id_0>')
    
    prefix = 'fill | '
    prefix=''
        
    train_data_en = train_features.copy()
    train_data_en = prefix + hearst_pattern + ' ' + train_data_en_data.term +  ' </s>'
    print(train_data_en.head())

    train_gold_en = train_target.copy()
    train_gold_en = train_gold_en.hypernym.str.split('\t').str.join(', ')
    print(train_gold_en.head())
    
    test_data_en = test_features.copy()
    test_data_en = prefix + hearst_pattern + ' ' + test_data_en.term +  ' </s>'
    print(test_data_en.head())

    test_gold_en = test_target.copy()
    test_gold_en = test_gold_en.hypernym.str.split('\t').str.join(', ')
    print(test_gold_en.head())
    
    return train_data_en, train_gold_en, test_data_en, test_gold_en


In [73]:
train_data_en, train_gold_en, test_data_en, test_gold_en = hearest_preprocessing(train_data_en_data, 
                                                                                 train_gold_en_data, 
                                                                                 test_data_en_data, 
                                                                                 test_gold_en_data, 
                                                                                 hearst_patterns[0])

0    There are a lot of <extra_id_0> such as blackf...
1    There are a lot of <extra_id_0> such as Turoni...
2    There are a lot of <extra_id_0> such as abhorr...
3    There are a lot of <extra_id_0> such as tropic...
4    There are a lot of <extra_id_0> such as milita...
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    There are a lot of <extra_id_0> such as malici...
1    There are a lot of <extra_id_0> such as buckle...
2    There are a lot of <extra_id_0> such as spelun...
3    There are a lot of <extra_id_0> such as quo wa...
4    There are a lot of <extra_id_0> such as Jeff F...
Name: term, dtype: object
0       malevolence, distaste, hatred, hate, malignity
1                                     

# Model

In [63]:
# use only 1 time per session
model_checkpoint = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=100, block_size=64)

In [69]:
# Input text
text = 'India is a <extra_id_0> of the world. </s>'

encoded = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
input_ids = encoded['input_ids'].to(device)

# Generaing 20 sequences with maximum length set to 5
outputs = model.generate(input_ids=input_ids, 
                          num_beams=200, num_return_sequences=20,
                          max_length=5)

_0_index = text.index('<extra_id_0>')
_result_prefix = text[:_0_index]
_result_suffix = text[_0_index+12:]  # 12 is the length of <extra_id_0>

def _filter(output, end_token='<extra_id_1>'):
    # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
    _txt = tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
    if end_token in _txt:
        _end_token_index = _txt.index(end_token)
        return _result_prefix + _txt[:_end_token_index] + _result_suffix
    else:
        return _result_prefix + _txt + _result_suffix

results = list(map(_filter, outputs))
results

['India is a part of the world. </s>',
 'India is a part of the world. </s>',
 'India is a different part of the world. </s>',
 'India is a small part of the world. </s>',
 'India is a melting pot of the world. </s>',
 'India is a big part of the world. </s>',
 'India is a very different part of the world. </s>',
 'India is a corner of the world. </s>',
 'India is a unique part of the world. </s>',
 'India is a beautiful part of the world. </s>',
 'India is a part of the world. </s>',
 'India is a huge part of the world. </s>',
 'India is a new part of the world. </s>',
 'India is a great part of the world. </s>',
 'India is a vibrant part of the world. </s>',
 'India is a vast part of the world. </s>',
 'India is a diverse part of the world. </s>',
 'India is a part of the world. </s>',
 'India is a large part of the world. </s>',
 'India is a powerhouse of the world. </s>']

In [111]:
import re
regex = "[a-zA-Z]+"


def _filter(output, end_token='<extra_id_1>'):
        # The first token is <unk> (inidex at 0) and the second token is <extra_id_0> (indexed at 32099)
        _txt = tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
        if end_token in _txt:
            _end_token_index = _txt.index(end_token)
            return _txt[:_end_token_index]
        else:
            return _txt

        
def predict_token(text):
    
    encoded = tokenizer.encode_plus(text, add_special_tokens=False, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)

    # Generaing 20 sequences with maximum length set to 10
    outputs = model.generate(input_ids=input_ids, 
                              num_beams=20, num_return_sequences=20,
                              max_length=10)

    _0_index = text.index('<extra_id_0>')
    _result_prefix = text[:_0_index]
    _result_suffix = text[_0_index+12:]  # 12 is the length of <extra_id_0>
    
    results = list(map(_filter, outputs))
    results = [test_string.replace(',' , '')for test_string in results]
    
    return results

for i in range(5):
    text = test_data_en.tolist()[i]
    print('pattern', text)
    results = predict_token(text)
    
    print('output', results)
    print()


pattern There are a lot of <extra_id_0> such as maliciousness </s>
output ['reasons', 'reasons for this', 'reasons', 'reasons for this', 'reasons', 'reasons', 'reasons for it', 'reasons', 'factors', 'reasons', 'reasons', 'reasons', 'reasons', 'reasons', 'other reasons', 'reasons', 'dangers', 'reasons', 'factors', 'factors']

pattern There are a lot of <extra_id_0> such as buckler </s>
output ['buckles', 'bucklers', 'buckles', 'belt buckles', 'buckles', 'accessories', 'buckler', 'different buckles', 'buckle accessories', 'buckles', 'options', 'options', 'buckle designs', 'accessories', 'accessories', 'options', 'options', 'buckle accessories', 'accessories', 'belt buckle']

pattern There are a lot of <extra_id_0> such as spelunker </s>
output ['online games', 'games', 'websites', 'games', 'sites', 'search engines', 'them', 'different games', 'games to play', 'games online', 'other games', 'games', 'fun games', 'games', 'websites', 'popular games', 'games available', 'games', 'sites', 't

# EVALUATION 

In [112]:
def predict(test_data_en, test_gold_en):
    
#   make predictions for each hyponyms
    test_pred_en=[]
    for text in tqdm(test_data_en.tolist()):
        pred_masked_token = predict_token(text)
        test_pred_en.append('\t'.join(pred_masked_token))
            
#   make txt format
    name  = 'pred_hearst_pattern.txt'

    test_pred_en_df = pd.DataFrame(test_pred_en)
    test_pred_en_df.to_csv(name, header=None, index=None)

In [113]:
%%time 

predict(test_data_en, test_gold_en)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [09:23<00:00,  2.66it/s]

CPU times: user 9min 22s, sys: 982 ms, total: 9min 23s
Wall time: 9min 23s


In [11]:
from contextlib import redirect_stdout
import io

In [12]:
def answers(str_ans, name_of_pattern):
    str_ans += name_of_pattern
    columns_name = []
    values = []
    for ind, metrics in enumerate(_std_out.split('\r\n')):
        
        if ind == 6:
            _name = 'name of hearst pattern'
            number = name_of_pattern
        else:
            _name, number = metrics.split(' ')
            number = round(float(number), 5)
            _name = _name[:-1]
        
        columns_name.append(_name)
        values.append([number])
        
    
        
    df = pd.DataFrame(values).T
    df.columns = columns_name
    df.set_index('name of hearst pattern', inplace=True)
    return df

In [17]:
name_of_pattern = hearst_patterns[0]
f = io.StringIO()
with redirect_stdout(f):
    !python scorer.py /home/jovyan/work/SemEval2018-Task9/test/gold/1A.english.test.gold.txt pred_hearst_pattern.txt

_std_out = f.getvalue()


_std_out = _std_out  + name_of_pattern
output = answers(_std_out, name_of_pattern)
output

,MRR,MAP,P@1,P@3,P@5,P@15
name of hearst pattern,,,,,,
There are a lot of [PARENT] such as,0.05528,0.02892,0.04133,0.028,0.02796,0.02821


# All hearst patterns table

In [129]:
import os

if 'hearst_patterns_result_table.csv' in os.listdir():
    df = pd.read_csv('hearst_patterns_result_table.csv')
    last_pattern = df['name of hearst pattern'].tail(1).values[0]
    
    ind = hearst_patterns.index(last_pattern)
    hearst_patterns_in_use = hearst_patterns[ind+1:]
    
else:
    hearst_patterns_in_use = hearst_patterns

In [132]:
hearst_patterns_table = []

for pattern in hearst_patterns_in_use:
    
#   Data processing
    train_data_en, train_gold_en, test_data_en, test_gold_en = hearest_preprocessing(train_data_en_data,
                                                                                     train_gold_en_data, 
                                                                                     test_data_en_data, 
                                                                                     test_gold_en_data, 
                                                                                     pattern)
#   Predict
    predict(test_data_en, test_gold_en)
    
#   Metrics
    
    f = io.StringIO()
    with redirect_stdout(f):
        !python debuged_task9-scorer.py SemEval2018-Task9/test/gold/1A.english.test.gold.txt pred_hearst_pattern.txt


    _std_out = f.getvalue()


    _std_out = _std_out  + pattern
    
    pattern_table = answers(_std_out, pattern)
    hearst_patterns_table.append(pattern_table)


    df = pd.concat(hearst_patterns_table)
    df.to_csv('hearst_patterns_result_table.csv')


0    There are a lot of <extra_id_0> here such as b...
1    There are a lot of <extra_id_0> here such as T...
2    There are a lot of <extra_id_0> here such as a...
3    There are a lot of <extra_id_0> here such as t...
4    There are a lot of <extra_id_0> here such as m...
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    There are a lot of <extra_id_0> here such as m...
1    There are a lot of <extra_id_0> here such as b...
2    There are a lot of <extra_id_0> here such as s...
3    There are a lot of <extra_id_0> here such as q...
4    There are a lot of <extra_id_0> here such as J...
Name: term, dtype: object
0       malevolence, distaste, hatred, hate, malignity
1                                     

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:58<00:00,  2.79it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0          Other <extra_id_0> such as blackfly </s>
1          Other <extra_id_0> such as Turonian </s>
2        Other <extra_id_0> such as abhorrence </s>
3    Other <extra_id_0> such as tropical storm </s>
4    Other <extra_id_0> such as militarization </s>
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    Other <extra_id_0> such as maliciousness </s>
1          Other <extra_id_0> such

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:32<00:00,  2.93it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0     My favorite <extra_id_0> is either blackfly </s>
1     My favorite <extra_id_0> is either Turonian </s>
2    My favorite <extra_id_0> is either abhorrence ...
3    My favorite <extra_id_0> is either tropical st...
4    My favorite <extra_id_0> is either militarizat...
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    My favorite <extra_id_0> is either maliciousne...
1      My favor

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0          <extra_id_0> for example blackfly </s>
1          <extra_id_0> for example Turonian </s>
2        <extra_id_0> for example abhorrence </s>
3    <extra_id_0> for example tropical storm </s>
4    <extra_id_0> for example militarization </s>
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    <extra_id_0> for example maliciousness </s>
1          <extra_id_0> for example buckler </

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:59<00:00,  2.78it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0          <extra_id_0> for instance blackfly </s>
1          <extra_id_0> for instance Turonian </s>
2        <extra_id_0> for instance abhorrence </s>
3    <extra_id_0> for instance tropical storm </s>
4    <extra_id_0> for instance militarization </s>
Name: term, dtype: object
0                           homopterous insect, insect
1    technical specification, geologic timescale, p...
2                      distaste, hatred, hate, disgust
3    atmosphere, windstorm, violent storm, air curr...
4                                       social control
Name: hypernym, dtype: object
0    <extra_id_0> for instance maliciousness </s>
1          <extra_id_0> for instance buc

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:34<00:00,  2.92it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [135]:
metrics_table = pd.read_csv('hearst_patterns_result_table.csv').set_index('name of hearst pattern')

In [136]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

def highlight_min(s, props=''):
    return np.where(s == np.nanmin(s.values), props, '')

metrics_table2 = metrics_table.style.apply(highlight_max, 
                                          props='color:white; background-color:#1FC29D', axis=0).apply(highlight_min, props='color:white; background-color:#FF5555', axis=0)
metrics_table2

,MRR,MAP,P@1,P@3,P@5,P@15
name of hearst pattern,,,,,,
There are a lot of [PARENT] here such as,0.001690,0.000710,0.000670,0.000890,0.000600,0.000910
Other [PARENT] such as,0.020890,0.013760,0.008670,0.011560,0.013100,0.015720
My favorite [PARENT] is either,0.168910,0.093550,0.114670,0.090110,0.086810,0.095720
There were a lot of [PARENT] here such as,0.001780,0.001190,0.000000,0.001000,0.001070,0.001600
which includes various [PARENT] like,0.007870,0.006150,0.001330,0.004110,0.004870,0.009190
Other [PARENT] especially,0.062890,0.035380,0.040000,0.029670,0.033300,0.037260
which includes various [PARENT] such as,0.004560,0.003770,0.000000,0.002890,0.003270,0.005400
My favorite [PARENT] is,0.180120,0.108980,0.114670,0.095330,0.094000,0.121550
I know such types of [PARENT] as,0.089240,0.059250,0.040000,0.049330,0.053100,0.068230
